# Examination Dynamics Study

This jupyter notebook shows how periodic examinations affect a pandemy evolution.

## Table of Contents:
* Model definition
* Import Libraries
* Single Simulation Study
    * Parameter Settings
    * SEIR plot
    * Accumulated Infected
    * New Daily Infected
    * Ammount of exams
* Sensitivity Analysis: 
    * Examination Rate
        * Peak Size
        * Peak time
        * Prevalence
    * Examination Period
        * Peak Size
        * Peak time
        * Prevalence
    * Examination Duty
        * Peak Size
        * Peak time
        * Prevalence  
    * Examination Accuracy
        * Peak Size
        * Peak time
        * Prevalence        
* Multidimensional Analysis:
    * Examination rate vs Examination periods
    * Examination Rate vs Duty


# Model Definition

\begin{equation} 
\frac{dS}{dt} = -\beta \alpha \frac{S I}{N+k_{I}I+k_{R}R}
\end{equation}

\begin{equation} 
\frac{dE}{dt} = \beta \alpha \frac{S I}{N+k_{I}I+k_{R}R} - \sigma E  
\end{equation}
\begin{equation} 
\frac{dI}{dt} = \sigma E - \gamma I - \lambda_{e} \sqcap(\omega_{e} t) P_{I}
\end{equation}
\begin{equation} 
\frac{dR}{dt} = \gamma I + \lambda_{e} \sqcap(\omega_{e} t) P_{I}
\end{equation}
\begin{equation} 
S_i+E_i+I_i+R_i = N_i
\end{equation}
\begin{equation} 
P_{I} = \frac{I}{N}
\end{equation}
\begin{equation} 
e(t) = \lambda_{e} \sqcap(\omega_{e} t)
\end{equation}
\begin{equation} 
e_{I}(t) = \lambda_{e} \sqcap(\omega_{e} t)
\end{equation}

## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

# Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows': 
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIRQ import SEIR
from Quarantine import Quarantine
from Quarantine import Exams

Linux


## Simulation Parameters

In [ ]:
beta = 0.15 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 500
# Population
population = 1000000
# Initial Active Infected 
I0 = 1000

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k_I=0
# Immunity Shield
k_R=0
# Quarantine
s1 = Quarantine(1)


testaccuracy = 0.9
# Proportion of effectively quarantined detected infected
lambda_Q=1
# Time between test and quarantine
T_T = 1 
# Time in quarantine
T_Q = 14
# Tracing: How many extra infected you can detect for each infected detected through random search
lambda_Tr=0

-------------------------------------

# Single simulation study

In [ ]:
examrate = 20000 # Persons per day
period = 1
duty = 1
psi = Exams(examrate,period,duty) # 

## Run simulation

In [ ]:
%%capture
# To show the cell's output comment the former line


simulation = SEIR(tsim=tsim,alpha=s1.alpha,psi = psi,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1,testaccuracy=testaccuracy,
                  lambda_Q=lambda_Q,T_T = T_T,T_Q = T_Q,lambda_Tr=lambda_Tr)
simulation.integr_sci(0,tsim,0.01)
# No exams dynamic for reference:
simulation_reference =  SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1,testaccuracy=testaccuracy)
simulation_reference.integr_sci(0,tsim,0.01)
print('simulation finished')

### SEIR Plot with Active infected

In [ ]:
plt.plot(simulation.t,simulation.S,label='S',color = 'blue')
plt.plot(simulation.t,simulation.E,label='E',color = 'cyan')
plt.plot(simulation.t,simulation.I,label='I',color = 'red')
plt.plot(simulation.t,simulation.R,label='R',color = 'green')
plt.plot(simulation.t,simulation.Q,label='Q',color = 'purple')
plt.plot(simulation_reference.t,simulation_reference.S,label='S - No exams',color = 'blue',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.E,label='E - No exams',color = 'cyan',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.I,label='I - No exams',color = 'red',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.R,label='R - No exams',color = 'green',linestyle='dashed')

plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

### Quarantine dynamics

In [ ]:
plt.plot(simulation.t,simulation.Q,label='Q',color = 'purple')
plt.plot(simulation.t,simulation.I_T,label='I_t',color = 'red')
plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

### Plot Accumulated Infected

In [ ]:
plt.plot(simulation.t,simulation.I_ac,label='I_ac with exams')
plt.plot(simulation_reference.t,simulation_reference.I_ac,label='I_ac No Exams', linestyle='dashed')
plt.title('Accumulated Infected')
plt.legend(loc=0)
plt.show()

### Plot New Daily Infected

In [ ]:
plt.plot(simulation.t,simulation.I_d,label='I_d with exams')
plt.plot(simulation_reference.t,simulation_reference.I_d,label='I_d no exams', linestyle='dashed')
plt.title('New Daily Infected')
plt.legend(loc=0)
plt.show()

### Amount of exams

In [ ]:
plt.plot(simulation.t,simulation.e,label='Exams')
plt.title('Amount of exams performed')
plt.legend(loc=0)
plt.show()
print('We performed '+str(int(np.round(simulation.e[-1])))+' exams in total')

-------------------------------------------

# Sensitivity Analysis: 

## Examination rate
All examination campaings will have 50% of examination time during the different time periods.

In [ ]:
beta = 0.15 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I0 = 1000

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k_I=0
# Immunity Shield
k_R=0
# Quarantine
s1 = Quarantine(1)

duty = 1
examrate = list(np.arange(0.1,5,0.1)*population/100)# Persons per day
period = 1

psi = [Exams(i,period,duty) for i in examrate] 

testaccuracy=0.9

### Run simulation

In [ ]:
%%capture
# To show the cell's output comment the former line

def simulate(simulation,tsim):
    simulation.integr_sci(0,tsim,0.1)
    return simulation

sims = []
for i in range(len(examrate)):        
    sims.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1,testaccuracy=testaccuracy))
    
num_cores = multiprocessing.cpu_count()
simulation = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],tsim) for i in range(len(sims)))

print('ready')

In [ ]:
# Calculate indicatros
peak = []
for i in range(len(examrate)):    
    peak.append(100*simulation[i].peak/population)
    
peaktime = []
for i in range(len(examrate)):
    peaktime.append(simulation[i].peak_t)        
    
prevalence = []
for i in range(len(examrate)):
    prevalence.append(simulation[i].prevalence_total[-1])    

### Peak Size

In [ ]:
plt.plot(examrate,peak,label='Infected during peak')
plt.title('Peak size (%) per examrate')
plt.legend(loc=0)
plt.ylim(0,np.max(peak)*1.1)
plt.show()

### Peak time

In [ ]:
plt.plot(examrate,peaktime,label='Time when peak occurs')
plt.title('Peak time shift')
plt.legend(loc=0)
plt.ylim(0,np.max(peaktime)*1.1)
plt.show()

### Prevalence

In [ ]:
plt.plot(examrate,prevalence,label='% Percentage of total population')
plt.title('Prevalence at the end of the simulation')
plt.legend(loc=0)
plt.ylim(0,np.max(prevalence)*1.1)
plt.show()

## Examination Period
All examination campaings will have 50% of examination time during the different time periods.

## Examination Duty
All examination campaings will have 50% of examination time during the different time periods.

## Examination Accuracy
All examination campaings will have 50% of examination time during the different time periods.

-------------------------

# Multidimensional Analysis

## Examination rate vs Examination periods
All examination campaings will have 50% of examination time during the different time periods.

In [ ]:
duty = 0.5

examrate = [1000,5000,10000,20000,50000] # Persons per day
period = [1,2,5,10,15,30]

psi = [[Exams(i,j,duty) for j in period] for i in examrate] 

time = np.arange(0,100,0.1)
exams = [[[psi[i][j](t) for t in time] for j in range(len(period))] for i in range(len(examrate))]

### Plot Exam campaings

In [ ]:
n = 3
time = np.arange(0,100,0.1)
exams = [[[psi[i][j](t) for t in time] for j in range(len(period))] for i in range(len(examrate))]
#plt.plot(time,exams[i][j])

fig, axs = plt.subplots(len(examrate), len(period))
for i in range(len(examrate)):
    for j in range(len(period)):
        axs[i,j].plot(time,exams[i][j],label='Examination Rate:'+str(examrate[i])+' Period: '+str(period[j]))
        axs[i,j].legend(loc=0)
    #axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))

### Simulate

In [ ]:
%%capture
# To show the cell's output comment the former line

# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i][j],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1))
    sims.append(aux)
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

## Period Analysis

### Period analysis over a single amount of exam rate

In [ ]:
j = 0
plt.plot(simulation_reference.t,simulation_reference.I_d,label='No exams',color = 'black',linestyle='dashed')
for i in range(len(period)):
    plt.plot(simulation[j][i].t,simulation[j][i].I_d,label='Examination Period: '+str(period[i]))
plt.legend(loc=0)
plt.title('New Daily Infected | '+str(examrate[j])+' exams/day')
plt.show()

### Multiple rates for each period

In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(examrate)))
n = 3
fig, axs = plt.subplots(int(len(period)/n), n)
for j in range(len(period)):
    axs[int(j/n),j%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for i in range(len(examrate)):
        axs[int(j/n),j%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination Period: '+str(period[j]))
        axs[int(j/n),j%n].legend(loc=0)
    axs[int(j/n),j%n].set_title('Period: '+str(period[j])+' days')


## Exam Analysis

### Exam analysis over a single amount of period

In [ ]:
j = -1
plt.plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
for i in range(len(examrate)):
    plt.plot(simulation[i][j].t,simulation[i][j].I_d,label='Exams per day: '+str(examrate[i]))
plt.legend(loc=0)
plt.title('New Daily Infected | Campaing periods: '+str(period[j]))
plt.show()

### Multiple periods for each exam rate

In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(period)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(period)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination Period: '+str(period[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))


## Contour plot

In [ ]:
duty = 0.5
population = 1000000
examrate = np.array([0.1,0.3,0.5,0.8,1,1.5,2,2.5,3,4,5])*population/100 # Persons per day
period = [1,5,10,20,30,40,50,60,70,80,90]

psi = [[Exams(i,j,duty) for j in period] for i in examrate] 

time = np.arange(0,100,0.1)
exams = [[[psi[i][j](t) for t in time] for j in range(len(period))] for i in range(len(examrate))]

In [ ]:
# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i][j],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1))
    sims.append(aux)
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))
simulation2 = simulation
print('ready')

In [ ]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):
        aux.append(100*simulation2[i][j].peak/population)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):
        aux.append(simulation2[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):    
        aux.append(simulation2[i][j].prevalence_total[-1])  
    prevalence.append(aux)

### Peak size proportion

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(period,examrate*100/population,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size Proportion')
ax.set_xlabel('Time period')
ax.set_ylabel('ExamRate')
plt.show() 

### Peak  time shift

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(period,examrate*100/population,peaktime) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak time')
ax.set_xlabel('Time period')
ax.set_ylabel('ExamRate')
plt.show() 

### SeroPrevalence

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(period,examrate*100/population,peaktime) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak time')
ax.set_xlabel('Time period')
ax.set_ylabel('ExamRate')
plt.show() 

----------------

# Examrate vs duty

In [ ]:
examrate = [1000,5000,10000,20000,50000] # Persons per day
period = 30
duty = [0.1,0.25,0.5,0.75,1]
psi = [[Exams(i,period,j) for j in duty] for i in examrate] 

In [ ]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(duty)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i][j],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1))
    sims.append(aux)

In [ ]:
%%capture
# To show the cell's output comment the former line

# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Reference simulation
simulation_reference =  SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1)
simulation_reference.integr_sci(0,tsim,0.01)
print('simulation finished')

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

In [ ]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(duty)):
        aux.append(100*simulation[i][j].peak/population)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(duty)):
        aux.append(simulation[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(duty)):    
        aux.append(simulation[i][j].prevalence_total[-1])  
    prevalence.append(aux)

## GridPlot

In [ ]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(duty)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(duty)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination duty: '+str(duty[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))


## Single plots

### Peak Size

In [ ]:
for j in range(len(examrate)):
    plt.plot(duty,np.array(peak).transpose()[j],label='Exam Rate: '+str(examrate[j]))
plt.legend(loc=1)
plt.ylabel('Relative Peak size (%)')
plt.xlabel('Duty')
plt.title('Duty Study: Peak Size')
plt.show()

### Peak time

In [ ]:
for j in range(len(examrate)):
    plt.plot(duty,np.array(peaktime)[j],label='Exam Rate: '+str(examrate[j]))
plt.legend(loc=1)
plt.ylabel('Peak day')
plt.xlabel('Duty')
plt.title('Duty Study: Peak time')
plt.show()

### Prevalence

In [ ]:
for j in range(len(examrate)):
    plt.plot(duty,np.array(prevalence).transpose()[j]*100,label='Exam Rate: '+str(examrate[j]))
plt.legend(loc=1)
plt.ylabel('Prevalence %')
plt.xlabel('Duty')
plt.title('Duty Study: Prevalence')
plt.show()

## Contour plots

### Peak Size

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(duty,np.array(examrate)*100/population,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak size')
ax.set_xlabel('Duty')
ax.set_ylabel('ExamRate')
plt.show() 

### Peak time

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(duty,np.array(examrate)*100/population,peaktime) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak time')
ax.set_xlabel('Duty')
ax.set_ylabel('ExamRate')
plt.show() 

### Seroprevalence

In [ ]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(duty,np.array(examrate)*100/population,prevalence) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Prevalence')
ax.set_xlabel('Duty')
ax.set_ylabel('ExamRate')
plt.show() 

# Test results delay analysis

In [23]:
beta = 0.15 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I0 = 1000

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k_I=0
# Immunity Shield
k_R=0
# Quarantine
s1 = Quarantine(1)

duty = 1
examrate = list(np.arange(0.1,5,0.1)*population/100)# Persons per day
period = 1

psi = [Exams(i,period,duty) for i in examrate] 

testaccuracy=0.9
T_T = [1,2,3,4,5]
lambda_Tr = 0

# Proportion of effectively quarantined detected infected
lambda_Q=1

# Time in quarantine
T_Q = 14


In [37]:
examrate = np.array([1000,5000,10000,20000,50000]) # Persons per day
period = 30
duty = 1
psi = [Exams(i,period,duty) for i in examrate] 

In [25]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i],beta=beta,mu=mu,I0=I0,population=population,expinfection=0,testaccuracy=testaccuracy,lambda_Q=lambda_Q,T_T = T_T[j],T_Q = T_Q,lambda_Tr=lambda_Tr))
    sims.append(aux)

In [26]:
#%%capture
# To show the cell's output comment the former line

# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

# Reference simulation
simulation_reference =  SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1)
simulation_reference.integr_sci(0,tsim,0.01)
print('simulation finished')

# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

simulation finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0532s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0522s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.1

In [27]:
# Calculate indicators
peak = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(100*simulation[i][j].peak/population)
    peak.append(aux)

    
peaktime = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):
        aux.append(simulation[i][j].peak_t)
    peaktime.append(aux)

prevalence = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(T_T)):    
        aux.append(simulation[i][j].prevalence_total[-1])  
    prevalence.append(aux)

## GridPlot

In [34]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(T_T)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination results delay: '+str(T_T[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))
fig.suptitle('New Daily Infected')

Text(0.5, 0.98, 'New Daily Infected')

In [33]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(T_T)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_T,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(T_T)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_T,label='Examination results delay: '+str(T_T[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))
fig.suptitle('Infected waiting for test results')

Text(0.5, 0.98, 'Infected waiting for test results')

In [39]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(T_T,examrate*100/population,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size Proportion')
ax.set_xlabel('Results Delay')
ax.set_ylabel('ExamRate')
plt.show() 